In [5]:
import keras.optimizers
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout
from keras.models import Model

In [6]:
data = tf.keras.utils.image_dataset_from_directory('data', image_size=(224,224), batch_size=8, label_mode='categorical')
data_iterator = data.as_numpy_iterator()

Found 14100 files belonging to 7 classes.


In [7]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

In [8]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [9]:
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False,input_shape=(224, 224, 3))

In [10]:
x = base_model.output

x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Flatten()(x)  # Flatten the output of the VGG16 base model
x = Dense(units=7, activation='softmax')(x)  # Add a Dense layer with softmax activation

In [11]:
model = Model(inputs=base_model.input, outputs=x)

In [29]:
for layer in base_model.layers:
    layer.trainable = False

In [12]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model.fit(train, validation_data=val, epochs=10)

Epoch 1/10
1234/1234 [==============================] - 66s 47ms/step - loss: 0.7524 - accuracy: 0.7020 - val_loss: 0.7272 - val_accuracy: 0.7159
Epoch 2/10
  65/1234 [>.............................] - ETA: 41s - loss: 0.5844 - accuracy: 0.7385

KeyboardInterrupt: 